In [1]:
import matplotlib.pyplot as plt
plt.style.use('default')
import numpy as np
import re
import nltk

# Aula 9: Análise Sintática
**Objetivo da aula**: ao fim desta aula, o aluno será capaz de desenhar gramáticas para a construção de árvores sintáticas em linguagem natural para acessar texto semanticamente

**AVISO IMPORTANTE**
Nesta aula, estou assumindo que todos os alunos já usaram gramáticas livres de contexto para fazer parsing e análise sintática em um contexto de disciplina de compiladores. Se você não faz ideia do que é uma gramática livre de contexto e um parser, use um dos muitos recursos online para estudar antes da aula. Algumas possibilidades para começar sua busca são [esta](https://www.geeksforgeeks.org/classification-of-context-free-grammars/) e [esta](https://en.wikipedia.org/wiki/Context-free_grammar).


# Exercício 1
*Objetivo: lembrar-se do que é uma gramática livre de contexto e uma árvore sintática*

Numa gramática livre de contexto arbitrária, temos as seguintes regras de produção:
* S -> A B
* A -> A A
* A -> a
* B -> A B C
* B -> b
* C -> c

1. Mostre a árvore sintática que gera a string `a a a b c`
1. A string `a b c` pode ser gerada por essa gramática?



# Exercício 2
*Objetivo: lembrar-se do que é sujeito, objeto e predicado*

Na frase: “o lobo soprou a casa”, encontre o sujeito, o verbo, o predicado e o objeto. Como essas estruturas se organizam entre si?

# Exercício 3
*Objetivo: aplicar o conceito de gramática de substituições para compor uma frase em português*

Uma gramática de substituições funciona da seguinte forma:
Partimos de uma string de símbolos que contém somente uma raiz (geralmente é chamada de S, mas aqui chamaremos de FRASE.
Temos uma série de regras que nos permitem substituir um dos símbolos por outro ou outros. Por exemplo, podemos ter:
* FRASE -> SUJEITO PREDICADO
* FRASE -> PREDICADO SUJEITO
Quando temos mais de uma regra, podemos gerar frases com qualquer uma delas.
Por exemplo, nosso símbolo FRASE poderia ter sido substituído por dois símbolos: 
`SUJEITO PREDICADO`

Daí, temos mais e mais regras:
* SUJEITO -> ARTIGO SUBSTANTIVO
* SUJEITO -> SUBSTANTIVO
* SUBSTANTIVO -> ‘lobo’
* ARTIGO -> ‘o’

1. Quais regras seriam necessárias para montar a frase “o lobo soprou a casa”, de forma que obrigatoriamente passamos por construções como sujeito, predicado e objeto?
1. Aplique as regras que você construiu (ou seja, a gramática) para montar a árvore sintática que permite analisar a frase “o lobo soprou a casa”.

# Exercício 4
**Objetivo: fazer parsing usando as ferramentas do NLTK**

Parsing é o processo que constrói a árvore sintática que gera uma frase, à partir de uma gramátca.

Para fazer parsing, precisamos de (i) definir uma gramática e (ii) definir um algoritmo para fazer parsing.

1. Como definimos nossa gramática para o NLTK? Como ela se compara com aquela que você definiu no exercício anterior?
1. Como passamos nossa gramática escrita como string para um formato acessível pelas ferramentas do NLTK? Que tipo de objeto é usado para isso?
1. Como definimos qual será o algoritmo a ser usado no processo de parsing?
1. Econtre o trecho em que executamos o algoritmo propriamente dito e imprimimos a árvore sintática. Como essa árvore se compara com a que você construiu no exercício 2?
1. Modifique o código para que ele imprima diretamente a árvore sintática, ao invés de usar o método `pretty_print()`. Que estruturas de dados são usadas para representar a árvore em Python?



In [2]:
gramatica = """
FRASE -> SUJEITO PREDICADO
SUJEITO -> ENTIDADE
PREDICADO -> V OBJETO
OBJETO -> ENTIDADE
ENTIDADE -> ART N
ART -> 'o' | 'a'
N -> 'lobo'
N -> 'casa'
V -> 'soprou'
"""

In [3]:
from nltk import CFG
from nltk.parse import RecursiveDescentParser

grammar = CFG.fromstring(gramatica)
parser = RecursiveDescentParser(grammar, trace=0)
print(parser.grammar())

Grammar with 10 productions (start state = FRASE)
    FRASE -> SUJEITO PREDICADO
    SUJEITO -> ENTIDADE
    PREDICADO -> V OBJETO
    OBJETO -> ENTIDADE
    ENTIDADE -> ART N
    ART -> 'o'
    ART -> 'a'
    N -> 'lobo'
    N -> 'casa'
    V -> 'soprou'


In [4]:
for p in parser.parse("o lobo soprou o lobo".split()):
   print(p)
   #  p.pretty_print()

(FRASE
  (SUJEITO (ENTIDADE (ART o) (N lobo)))
  (PREDICADO (V soprou) (OBJETO (ENTIDADE (ART o) (N lobo)))))


# Exercício 5
*Objetivo: relacionar a análise sintática com perguntas sobre o conteúdo do texto*

Em uma frase que tem um sujeito, um verbo na voz ativa e um objeto (use a frase “o lobo soprou a casa” como exemplo), referencie construções sintáticas para responder:

1. Como podemos saber quem é o sujeito?
1. Quem realizou a ação descrita no verbo?
1. Como podemos saber o que o sujeito fez?
1. Sobre quem o sujeito aplicou a ação descrita no verbo?


# Exercício 6
*Objetivo: navegar pela árvore sintática gerada pelo parser para responder perguntas quanto ao conteúdo do texto*

O código abaixo busca responder à pergunta: “quem soprou a casa”.
1. Qual é a ideia subjacente que usamos para responder à pergunta?
1. Como podemos encontrar o sujeito da frase?
1. Seguindo essa mesma ideia, escreva o trecho de código que permita responder automaticamente às perguntas: “o que o lobo fez?” e “o que o lobo soprou?”.
1. Modifique a gramática e o código para gerar um sistema que analisa a frase: "o jogador chutou a bola" e seja capaz de responder a perguntas equivalentes sobre a frase.

In [5]:
# Pergunta: quem soprou a casa?
trees = []
for t in parser.parse("o lobo soprou a casa".split()):
    trees.append(t)


for subtree in trees[0].subtrees(): # Generate all subtrees
    if subtree.label()=='SUJEITO':
        print("Quem fez a ação? ", ' '.join(subtree.leaves()))

# Pergunta: o que o lobo fez?
for subtree in trees[0].subtrees(): # Generate all subtrees
    if subtree.label()=='PREDICADO':
        print("O que o Lobo fez? ", ' '.join(subtree.leaves()))



# Pergunta: o que o lobo soprou?
for subtree in trees[0].subtrees(): # Generate all subtrees
    if subtree.label()=='OBJETO':
        print("O que o lobo soprou? ", ' '.join(subtree.leaves()))



Quem fez a ação?  o lobo
O que o Lobo fez?  soprou a casa
O que o lobo soprou?  a casa


# Exercício 6
*Objetivo: entender como gramáticas livres de contexto probabilísticas podem ajudar a resolver ambiguidades*

Em português, podemos ter várias inversões sintáticas. Por exemplo: “ouviram do Ipiranga as margens plácidas” é gramaticalmente válido, assim como “as margens plácidas do Ipiranga ouviram”. Isso não significa, porém, que todas as variações de colocações sejam igualmente prováveis. Por exemplo: na linguagem cotidiana, é muito mais comum termos:
* FRASE -> SUJEITO PREDICADO
* PREDICADO -> VERBO OBJETO
que
* FRASE -> PREDICADO SUJEITO
* PREDICADO -> OBJETO VERBO
Ou seja: 

“Carlos viajou para o Caribe”

É bem mais comum que

“Para o Caribe viajou Carlos”

Podemos resolver esse tipo de ambiguidade de forma computacional, atribuindo probabilidades às produções de uma gramática. O código abaixo resolve essa ambiguidade atribuindo probabilidades às produções de uma gramática.

1. Qual é a ambiguidade sintática que temos na frase “ele entrou na loja de calças”?
1. Quais regras de produção da gramática modelam essa ambiguidade?
1. Quais foram as árvores sintáticas geradas? Qual é a probabilidade de cada uma delas ter sido gerada pela gramática?
1. De acordo com nossa gramática, qual é a interpretação mais provável para esta frase?
1. Modifique a gramática de forma a inverter qual é a interpretação mais provável da frase.



In [6]:
from nltk import PCFG
from nltk.parse import InsideChartParser    


gramatica = """
FRASE -> SUJEITO PREDICADO [1]
SUJEITO -> ENTIDADE [1]
PREDICADO -> V OBJETO [0.0005] | V OBJETO ADV [0.9995]
OBJETO -> ENTIDADE [0.5] | PREP_ART ENTIDADE [0.5]
ADV -> PREP ENTIDADE [1]
ENTIDADE -> N [0.25] | PROPESS [0.25] | N ADJ [0.5]
ADJ -> PREP N [1]
PROPESS -> 'ele' [1]
V -> 'entrou' [1]
PREP_ART -> 'na' [1]
N -> 'loja' [0.5]
PREP -> 'de' [1]
N -> 'calças' [0.5]
"""

grammar = PCFG.fromstring(gramatica)
parser = InsideChartParser(grammar, trace=0)
print(parser.grammar())

Grammar with 17 productions (start state = FRASE)
    FRASE -> SUJEITO PREDICADO [1.0]
    SUJEITO -> ENTIDADE [1.0]
    PREDICADO -> V OBJETO [0.0005]
    PREDICADO -> V OBJETO ADV [0.9995]
    OBJETO -> ENTIDADE [0.5]
    OBJETO -> PREP_ART ENTIDADE [0.5]
    ADV -> PREP ENTIDADE [1.0]
    ENTIDADE -> N [0.25]
    ENTIDADE -> PROPESS [0.25]
    ENTIDADE -> N ADJ [0.5]
    ADJ -> PREP N [1.0]
    PROPESS -> 'ele' [1.0]
    V -> 'entrou' [1.0]
    PREP_ART -> 'na' [1.0]
    N -> 'loja' [0.5]
    PREP -> 'de' [1.0]
    N -> 'calças' [0.5]


In [7]:
for p in parser.parse("ele entrou na loja de calças".split()):
    p.pretty_print()
    print("Probability:", p.prob())

                 FRASE                                     
    _______________|________________                        
   |                            PREDICADO                  
   |        ________________________|___________            
SUJEITO    |             OBJETO                ADV         
   |       |        _______|________        ____|_____      
ENTIDADE   |       |             ENTIDADE  |       ENTIDADE
   |       |       |                |      |          |     
PROPESS    V    PREP_ART            N     PREP        N    
   |       |       |                |      |          |     
  ele    entrou    na              loja    de       calças 

Probability: 0.0019521484375
                 FRASE                                       
    _______________|_________                                 
   |                     PREDICADO                           
   |        _________________|_______                         
   |       |                       OBJETO              

# Exercício 7
*Objetivo: relacionar os nós-folhas da árvore sintática a PoS-tagging*

A passagem para os nós-folha das árvores sintáticas que fizemos até o momento, na verdade, aplica uma funcionalidade que já conhecemos bem.

Qual é essa funcionalidade?

# Exercício 9
*Objetivo: usar uma estratégia de parsing baseada em expressões regulares*

Uma outra estratégia de Parsing é definir expressões regulares baseadas em PoS tags. Veja o exemplo no código abaixo.

1. Para que usamos o corpus Macmorpho?
1. Como agrupamos PoS tags?
1. Consulte a documentação do NLTK. Como a sintaxe de expressões regulares é usada pelo RegexParser?
1. Qual das expressões regulares é equivalente à produção “ENTIDADE -> ART N”?
1. Como a árvore sintática gerada se compara com as árvores geradas anteriormente?
1. Em que ordem as regras das expressões regulares são aplicadas?
1. O que acontece se encontrarmos alguma situação que não é contemplada por uma regra?
1. Modifique o código para que ele deixe de montar uma árvore sintática completa e somente encontre as entidades (artigo ou preposição + substantivo) presentes na frase de entrada.
1. Modifique a sua definição de entidades para que ela funcione na frase “O Rio de Janeiro continua lindo”.
1. A estratégia de parsing com expressões regulares é altamente dependente de uma etapa anterior. Que etapa é essa, e quão frágil é o parsing em relação a erros nessa etapa?



In [8]:
# Carregar o corpus e pré-processar
s = open('./datasets/macmorpho-train.txt', 'r', encoding='utf-8').read()
s = re.split(r'\.+_PU', s)
s = [s0.strip() for s0 in s]
s = [re.split('\s+', s0) for s0 in s]
s = [ [ tuple(re.split('_', w0)) for w0 in p] for p in s]
s = [ [ w for w in p if len(w)==2 ] for p in s ] 
s = [ [ (w[0].lower(), w[1]) for w in p] for p in s ]
s = [p for p in s if len(p)>5]      
# s[frase][palavra] = (palavra, tipo)
#print(s[0:2])

In [9]:
from nltk.tag import NgramTagger, DefaultTagger
taggers = []
taggers.append(DefaultTagger('N'))

for n in range(3):
    taggers.append(NgramTagger(n+1, s, backoff=taggers[-1]))

In [10]:
pos = taggers[-1].tag("o porquinho fugiu para a fazenda".split())
print(pos)

[('o', 'ART'), ('porquinho', 'N'), ('fugiu', 'V'), ('para', 'PREP'), ('a', 'ART'), ('fazenda', 'N')]


In [11]:
gramatica = ('''
    ENTIDADE: {<ART><N>}
    PREDICADO: {<V><PREP>?<ENTIDADE>}
    SUJEITO: {<ENTIDADE>}
    FRASE: {<SUJEITO><PREDICADO>}
    ''')
parser = nltk.RegexpParser(gramatica)
#print(parser)
for p in parser.parse(pos):
    print(p.pretty_print())


                            FRASE                                    
          ____________________|________                               
      SUJEITO                      PREDICADO                         
         |                     ________|______________                
      ENTIDADE                |        |           ENTIDADE          
   ______|__________          |        |        ______|_________      
o/ART          porquinho/N fugiu/V para/PREP a/ART          fazenda/N

None


# Exercício 10
*Objetivo: aplicar chunking para caracterizar personagens em um texto*

Na língua, usamos pronomes de tratamento para especificar a posição social da pessoa à qual nos referimos. A importância disso é diferente em cada sociedade e em cada situação. Na língua inglesa, por exemplo, usamos:
* Mr. para se referir a homens (casados ou não)
* Mrs. para se referir a mulheres casadas
* Miss para se referir a mulheres solteiras

Os pronomes de tratamento são especialmente importantes para caracterizar as personagens de "orgulho e preconceito", de Jane Austin (https://www.gutenberg.org/ebooks/42671). Faça um programa que classifique todos personagens do livro como homens, mulheres casadas ou mulheres solteiras.

In [27]:
from nltk import pos_tag

book = [nltk.corpus.gutenberg.raw(f) for f in nltk.corpus.gutenberg.fileids()][0]
book = re.sub("\[|\]|\(|\)|\n|\-|\.|\_|\;|\,", " ", book)
book = re.split("\s+", book.lower())

In [28]:
pos_book = pos_tag(book, lang="eng", tagset='universal')
del pos_book[0]

In [29]:
gramatica = ('''
    ENTIDADE: {<DET><NOUN>}
    PREDICADO: {<VERB><ADP>?<ENTIDADE>}
    SUJEITO: {<ENTIDADE>}
    FRASE: {<SUJEITO><PREDICADO>}
    ''')
parser = nltk.RegexpParser(gramatica)
#print(parser)
tree = parser.parse(pos_book)

In [30]:
print(s.leaves())

[('emma', 'VERB'), ('by', 'ADP'), ('jane', 'NOUN'), ('austen', 'ADJ'), ('1816', 'NUM'), ('volume', 'NOUN'), ('i', 'VERB'), ('chapter', 'NOUN'), ('i', 'NOUN'), ('emma', 'VERB'), ('woodhouse', 'ADP'), ('handsome', 'ADJ'), ('clever', 'NOUN'), ('and', 'CONJ'), ('rich', 'ADJ'), ('with', 'ADP'), ('a', 'DET'), ('comfortable', 'ADJ'), ('home', 'NOUN'), ('and', 'CONJ'), ('happy', 'ADJ'), ('disposition', 'NOUN'), ('seemed', 'VERB'), ('to', 'PRT'), ('unite', 'VERB'), ('some', 'DET'), ('of', 'ADP'), ('the', 'DET'), ('best', 'ADJ'), ('blessings', 'NOUN'), ('of', 'ADP'), ('existence', 'NOUN'), ('and', 'CONJ'), ('had', 'VERB'), ('lived', 'VERB'), ('nearly', 'ADV'), ('twenty', 'ADJ'), ('one', 'NUM'), ('years', 'NOUN'), ('in', 'ADP'), ('the', 'DET'), ('world', 'NOUN'), ('with', 'ADP'), ('very', 'ADV'), ('little', 'ADJ'), ('to', 'PRT'), ('distress', 'VERB'), ('or', 'CONJ'), ('vex', 'VERB'), ('her', 'PRON'), ('she', 'PRON'), ('was', 'VERB'), ('the', 'DET'), ('youngest', 'ADJ'), ('of', 'ADP'), ('the', 'DE